In [3]:
### LOAD NEEDED DATA ###
import pickle
import pandas as pd

with open('model_pickle', 'rb') as f:
    newGrid = pickle.load(f)
    
with open('target_encoder_pickle', 'rb') as f:
    target_encoder = pickle.load(f)
    
with open('full_pipeline_pickle', 'rb') as f:
    full_pipeline = pickle.load(f)
    
with open('wrongValues_pickle', 'rb') as f:
    wrongValues = pickle.load(f)
    
### load new data ###
def load_data2():
    
    f=open("kddcup.names.txt",'r')
    data=f.readlines()[1:]

    column_names=[]
    for line in data:
        column_names.append(line.split(':')[0])

    column_names.append('target')
    f.close()

    f=open("training_attack_types2.txt","r")
    data=f.readlines()

    attack_type_mapping={'normal':'normal'}
    for line in data:
        k,v=line.split()
        attack_type_mapping[k]=v

    f.close()
    
    ### read first 10000 rows
    temp=pd.read_csv('kddcup.data.gz',names=column_names)
    attack_type_mapping.keys()
    #print(attack_type_mapping)
    #for key, value in attack_type_mapping.items() :
    #    print (key, value)
    temp['target']=temp.target.apply(lambda r: attack_type_mapping[r[:-1]])
    
    return temp

newSet = load_data2()
newSet.drop_duplicates(inplace=True)


single_XX = newSet.drop(['target','num_root','srv_serror_rate','srv_rerror_rate','dst_host_srv_serror_rate','dst_host_srv_rerror_rate','dst_host_serror_rate','dst_host_rerror_rate','dst_host_same_srv_rate','num_outbound_cmds','is_host_login'],axis=1)
single_YY = newSet['target'].copy()
single_XX_prepared = full_pipeline.transform(single_XX)
single_YY = target_encoder.transform(single_YY)


test_prediction=newGrid.best_estimator_.predict(single_XX_prepared)

from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix

print(accuracy_score(single_YY,test_prediction))
print(precision_score(single_YY,test_prediction,average='weighted'))
print(recall_score(single_YY,test_prediction,average='weighted'))

0.9896091380049731
0.9909996330114459
0.9896091380049731
